<a href="https://colab.research.google.com/github/prax1807/Git-to-do/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extracting the Data from Spam Assasin

In [10]:
# prompt: unzip tar.bz file
import os
import urllib.request
import tarfile

urllib.request.urlretrieve("https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2", "ham.tar.bz2")
urllib.request.urlretrieve("https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2", "spam.tar.bz2")

# Extract the contents of the tar.bz2 file
with tarfile.open("ham.tar.bz2", "r:bz2") as tar:
    tar.extractall()

with tarfile.open("spam.tar.bz2", "r:bz2") as tar:
    tar.extractall()

os.rename('easy_ham', 'ham')

In [16]:

os.listdir(os.getcwd())

['.config',
 '.ipynb_checkpoints',
 'ham',
 'spam.tar.bz2',
 'spam',
 'ham.tar.bz2',
 'sample_data']

In [19]:
import os
import pandas as pd
def load_emails(directory):
    emails = []
    labels = []
    for label in ['spam', 'ham']:
        folder = os.path.join(directory, label)
        for filename in os.listdir(folder):
            with open(os.path.join(folder, filename), 'r', encoding='latin1') as file:
                emails.append(file.read())
                labels.append(label)
    return emails, labels

email,labels = load_emails('/content/')

In [23]:
labels[0:5]

['spam', 'spam', 'spam', 'spam', 'spam']

In [ ]:
import nltk